# K-Maxoids Clustering
Example comparing k-means and k-maxoids clustering methods.

K-maxoids automatically preserves extreme periods by selecting points closest to the convex hull.

Author: Maximilian Hoffmann

Import pandas and the relevant time series aggregation class

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import pandas as pd

# Configure Plotly for sphinx/nbsphinx output
import plotly.io as pio

import tsam
from tsam import ClusterConfig

pio.renderers.default = "notebook"

# Ensure results directory exists
RESULTS_DIR = Path("results")
RESULTS_DIR.mkdir(exist_ok=True)

### Input data 

Read in time series from testdata.csv with pandas

In [ ]:
raw = pd.read_csv("testdata.csv", index_col=0)

Show a slice of the dataset

In [ ]:
raw.head()

Show the shape of the raw input data: 4 types of timeseries (GHI, Temperature, Wind and Load) for every hour in a year

In [ ]:
raw.shape

Create a plot function for the temperature for a visual comparison of the time series

In [ ]:
# Use tsam's built-in plotting with plotly
# tsam.plot.heatmap(data, column, period_hours) creates interactive heatmaps

Plot an example series - in this case the temperature

In [ ]:
# Original temperature heatmap
tsam.plot.heatmap(raw, column="T", period_hours=24, title="Original Temperature")

### Simple k-mean aggregation

Initialize an aggregation class object with k-mean as method for eight typical days, without any integration of extreme periods. Alternative clusterMethod's are 'averaging','hierarchical', 'k_medoids' and 'k_maxoids'.

In [ ]:
result = tsam.aggregate(
    raw,
    n_periods=8,
    period_hours=24,
    cluster=ClusterConfig(method="kmeans"),
)

Create the typical periods

In [ ]:
typical_periods = result.typical_periods

Show shape of typical periods: 4 types of timeseries for 8*24 hours

In [ ]:
typical_periods.shape

Save typical periods to .csv file

In [ ]:
typical_periods.to_csv(RESULTS_DIR / "testperiods_kmeans.csv")

Repredict the original time series based on the typical periods

In [ ]:
reconstructed = result.reconstruct()

Plot the repredicted data

In [ ]:
# K-means predicted temperature heatmap
tsam.plot.heatmap(
    reconstructed, column="T", period_hours=24, title="K-means Predicted Temperature"
)

As seen, they days with the minimal temperature are excluded. In case that they are required they can be added to the aggregation as follow.

### k-maxoids aggregation including extreme periods

Initialize a time series aggregation based on k-maxoids, which automatically searches for points closest to the convex hull.

In [ ]:
result_maxoids = tsam.aggregate(
    raw,
    n_periods=8,
    period_hours=24,
    cluster=ClusterConfig(method="kmaxoids"),
    rescale=False,
)

Create the typical periods

In [ ]:
typical_periods_maxoids = result_maxoids.typical_periods

The aggregation can also be evaluated by indicators

In [ ]:
result_maxoids.accuracy

Repredict the original time series based on the typical periods

In [ ]:
reconstructed_maxoids = result_maxoids.reconstruct()

Plot repredicted data

In [ ]:
# K-maxoids predicted temperature heatmap
tsam.plot.heatmap(
    reconstructed_maxoids,
    column="T",
    period_hours=24,
    title="K-maxoids Predicted Temperature",
)

Here bigger biggest values and lower lowest values can be observed compared to k-means clustering.

### Comparison of the aggregations 
It was shown for the temperature, but both times all four time series have been aggregated. Therefore, we compare here also the duration curves  of the electrical load for the original time series, the aggregation with k-mean, and the k-maxoids aggregation.

In [ ]:
# Duration curve comparison using built-in function
tsam.plot.compare_results(
    {
        "Original": raw,
        "8 typ days (Centroids)": reconstructed,
        "8 typ days (Maxoids)": reconstructed_maxoids,
    },
    column="Load",
    plot_type="duration_curve",
)

Or as unsorted time series for an example week

In [ ]:
# Time slice comparison using built-in function
tsam.plot.compare_results(
    {
        "Original": raw,
        "8 typ days (Centroids)": reconstructed,
        "8 typ days (Maxoids)": reconstructed_maxoids,
    },
    column="Load",
    plot_type="time_slice",
    start="20100210",
    end="20100218",
)